# Italian Word Lemmatizer

### Import


In [247]:
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Bidirectional, TimeDistributed, RepeatVector, Activation, Dot, Lambda, Dropout, Add, Multiply, Masking
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import gensim
import tensorflow_addons as tfa
import tensorflow as tf
import pandas as pd
import datetime
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# set all random seeds
np.random.seed(42)
tf.random.set_seed(42)


## Dataset

In [260]:

dataset_path = "./dev.csv"
df_dev = pd.read_csv(dataset_path, sep="\t", header=None,
                     names=["word", "tag", "lemm"])

dataset_path = "./test.csv"
df_test = pd.read_csv(dataset_path, sep="\t", header=None,
                      names=["word", "tag", "lemm"])

df_dev["word"] = df_dev["word"].astype(str)
df_dev["tag"] = df_dev["tag"].astype(str)
df_dev["lemm"] = df_dev["lemm"].astype(str)

df_test["word"] = df_test["word"].astype(str)
df_test["tag"] = df_test["tag"].astype(str)
df_test["lemm"] = df_test["lemm"].astype(str)

# remove head
df_dev = df_dev.iloc[1:]
df_test = df_test.iloc[1:]

# removing rows where tag is nan
df_dev = df_dev.dropna(subset=["tag"])
df_dev = df_dev[df_dev["tag"] != "nan"]
df_test = df_test.dropna(subset=["tag"])
df_test = df_test[df_test["tag"] != "nan"]

# lower case all words
df_test["word"] = df_test["word"].str.lower()
df_dev["word"] = df_dev["word"].str.lower()


def get_sentences(df):
    words = []
    tags = []
    lemmas = []
    sentence = []
    max_s = 0
    for index, row in df.iterrows():
        word = row["word"]
        tag = row["tag"]
        lemm = row["lemm"]
        sentence.append([word, tag, lemm])

        if row["word"] in [".", "?", "!", ";"]:
            words.append([word for word, tag, lemm in sentence])
            tags.append([tag for word, tag, lemm in sentence])
            lemmas.append([lemm for word, tag, lemm in sentence])
            max_s = max(max_s, len(sentence))
            sentence = []

    print("Max sentence length: ", max_s)
    return words, tags, lemmas

# _s is for string
dev_words_s, dev_tags_s, dev_lemmas_s = get_sentences(df_dev)
test_words_s, test_tags_s, test_lemmas_s = get_sentences(df_test)
print("Number of sentences in dev set: ", len(dev_words_s))
print("Number of sentences in test set: ", len(test_words_s))

for i in range(len(dev_words_s)):
    if len(dev_words_s[i]) != len(dev_tags_s[i]) or len(dev_words_s[i]) != len(dev_lemmas_s[i]):
        print("Dimension mismatch in sentence: ", i)
        print("Words: ", dev_words_s[i])
        print("Tags: ", dev_tags_s[i])
        print("Lemmas: ", dev_lemmas_s[i])
        break


Max sentence length:  95
Max sentence length:  107
Number of sentences in dev set:  703
Number of sentences in test set:  5596


## Word Context

In [261]:
CTX_DIM = 13
PRE_VALUE = "<PRE>"
POST_VALUE = "<POST>"

def get_context(words, tags, lemmas):
    ctx = []
    w = []
    tag = []
    lemma = []

    for s_index in range(len(words)):
        s = words[s_index]
        sentence = " ".join(s)
        s = [PRE_VALUE] * CTX_DIM + s + [POST_VALUE] * CTX_DIM

        for w_index in range(len(s)):
            if w_index < CTX_DIM or w_index >= len(s) - CTX_DIM:
                continue

            context = s[w_index - CTX_DIM:w_index] + s[w_index + 1:w_index  + CTX_DIM + 1]
            context = " ".join(context)
            ctx.append(context)
            w.append(words[s_index][w_index-CTX_DIM])
            tag.append(tags[s_index][w_index-CTX_DIM])
            lemma.append(lemmas[s_index][w_index-CTX_DIM])

    return ctx, w, tag, lemma

dev_ctx, dev_words, dev_tags, dev_lemmas = get_context(dev_words_s, dev_tags_s, dev_lemmas_s)
test_ctx, test_words, test_tags, test_lemmas = get_context(test_words_s, test_tags_s, test_lemmas_s)

### Example of context

In [250]:
print("CTX Dim:", CTX_DIM, "\n")
for i in range(3):
    index = np.random.randint(0, len(dev_ctx))
    print("Context: ", dev_ctx[index])
    print("Word: ", dev_words[index])
    print("Tag: ", dev_tags[index])
    print("Lemma: ", dev_lemmas[index])
    print()


CTX Dim: 13 

Context:  , e le sorelle che non trovavano marito neanche a regalarle , e mamma la quale filava al buio per risparmiar l' olio della lucerna ,
Word:  la
Tag:  art
Lemma:  la

Context:  esiste una grave frattura tra gli stati uniti e altre grandi potenze locali internazionali , egli sfider&agrave; apertamente le risoluzioni onu provocando la reazione militare statunitense
Word:  e
Tag:  conj_c
Lemma:  e

Context:  <PRE> <PRE> <PRE> <PRE> <PRE> <PRE> <PRE> <PRE> <PRE> era una bandiera che tempo aveva cessato di sventolare . <POST> <POST> <POST> <POST> <POST> <POST> <POST>
Word:  da
Tag:  prep
Lemma:  da



## Open Class Words
The evaluation is done only on open-class words and not to functional words: only the tokens having a PoS-tag comprised in the set ADJ *, ADV, NN, V * had to be lemmatised, in all the other cases the token could be copied unchanged into the lemma column as they were not considered for the evaluation (the asterisk indicates all PoS-tag possibilities beginning with that prefix).

In [263]:
def get_open_class_words(ctx, words, tags, lemmas):
    open_class_words = []
    open_class_ctx = []
    open_class_tags = []
    open_class_lemmas = []

    for i in range(len(words)):
        if "adj_" in tags[i] or "adv" in tags[i] or "nn" in tags[i] or "v_" in tags[i]:
            open_class_words.append(words[i])
            open_class_ctx.append(ctx[i])
            open_class_tags.append(tags[i])
            open_class_lemmas.append(lemmas[i])

    return open_class_ctx, open_class_words, open_class_tags, open_class_lemmas


test_ctx, test_words, test_tags, test_lemmas = get_open_class_words(test_ctx, test_words, test_tags, test_lemmas)

test_ctx:  62597
test_words:  62597
test_tags:  62597
test_lemmas:  62597


## Encoding

In [270]:
# word encoder
word_tokenizer = Tokenizer(filters="")
word_tokenizer.fit_on_texts(dev_ctx + test_ctx)

# tag encoder
tag_tokenizer = Tokenizer(filters="")
tag_tokenizer.fit_on_texts(dev_tags_s + test_tags_s)

# lemma encoder
lemma_tokenizer = Tokenizer(filters="")
lemma_tokenizer.fit_on_texts(dev_lemmas_s + test_lemmas_s)

dev_ctx_e = word_tokenizer.texts_to_sequences(dev_ctx)
test_ctx_e = word_tokenizer.texts_to_sequences(test_ctx)

PRE_E = word_tokenizer.texts_to_sequences([PRE_VALUE])[0][0]
POST_E = word_tokenizer.texts_to_sequences([POST_VALUE])[0][0]


dev_tags_e = tag_tokenizer.texts_to_sequences(dev_tags)
test_tags_e = tag_tokenizer.texts_to_sequences(test_tags)

# get all unique letter in words
characters = set()

for lemma in df_dev["lemm"].unique():
    for letter in lemma:
        characters.add(letter)

for lemma in df_test["lemm"].unique():
    for letter in lemma:
        characters.add(letter)

# add padding and unknown to characters
characters.add(" ")

# the length of the vocab for one-hot encoded char
VOCAB_SIZE = len(characters)

print ("Vocab size: ", VOCAB_SIZE)
# order characters
characters = sorted(list(characters))


char2idx = {char: idx for idx, char in enumerate(characters)}
idx2char = {idx: char for idx, char in enumerate(characters)}

MAX_WORD_LENGTH = 0
for w in dev_words + test_words + dev_lemmas + test_lemmas:
    MAX_WORD_LENGTH = max(MAX_WORD_LENGTH, len(w))
MAX_WORD_LENGTH += 1
print("Max word length: ", MAX_WORD_LENGTH)

def encode_words(words):
    encoded_words = []
    for word in words:
        word_e = []
        for letter in word:
            word_e.append(characters.index(letter))
        encoded_words.append(word_e)
    return encoded_words

dev_words_e = encode_words(dev_words)
test_words_e = encode_words(test_words)
dev_lemmas_e = encode_words(dev_lemmas)
test_lemmas_e = encode_words(test_lemmas)

dev_words_e = tf.keras.preprocessing.sequence.pad_sequences(dev_words_e, maxlen=MAX_WORD_LENGTH, padding="post")
test_words_e = tf.keras.preprocessing.sequence.pad_sequences(test_words_e, maxlen=MAX_WORD_LENGTH, padding="post")

dev_lemmas_e = tf.keras.preprocessing.sequence.pad_sequences(dev_lemmas_e, maxlen=MAX_WORD_LENGTH, padding="post")
test_lemmas_e = tf.keras.preprocessing.sequence.pad_sequences(test_lemmas_e, maxlen=MAX_WORD_LENGTH, padding="post")


# show random data point
index = np.random.randint(0, len(dev_ctx))
print("Context: ", dev_ctx[index], " -> ", dev_ctx_e[index])
print("Words: ", dev_words[index], " -> ", dev_words_e[index])
print("Tag: ", dev_tags[index], " -> ", dev_tags_e[index])
print("Lemma: ", dev_lemmas[index], " -> ", dev_lemmas_e[index])

Vocab size:  60
Max word length:  26
Context:  leader delle confederazioni sindacali prima_di rientrare a varsavia , ieri ha completato i con i governanti italiani vedendo il vice presidente del consiglio claudio martelli .  ->  [609, 40, 7058, 4131, 338, 6304, 10, 4566, 3, 121, 26, 4567, 18, 23, 18, 6522, 292, 2304, 8, 3858, 157, 15, 635, 3590, 9561, 6]
Words:  colloqui  ->  [36 48 45 45 48 50 54 42  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0]
Tag:  nn  ->  [1]
Lemma:  colloquio  ->  [36 48 45 45 48 50 54 42 48  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0]


In [271]:
# one hot encode the characters for the lemmas
dev_lemmas_e = tf.one_hot(dev_lemmas_e, VOCAB_SIZE)
test_lemmas_e = tf.one_hot(test_lemmas_e, VOCAB_SIZE)

In [272]:
# trnasform to numpy array
dev_ctx_e = np.array(dev_ctx_e)
dev_words_e = np.array(dev_words_e)
dev_tags_e = np.array(dev_tags_e)
dev_lemmas_e = np.array(dev_lemmas_e)

test_ctx_e = np.array(test_ctx_e)
test_words_e = np.array(test_words_e)
test_tags_e = np.array(test_tags_e)
test_lemmas_e = np.array(test_lemmas_e)

print("Context shape: ", dev_ctx_e.shape)
print("Words shape: ", dev_words_e.shape)
print("Tags shape: ", dev_tags_e.shape)
print("Lemmas shape: ", dev_lemmas_e.shape)

Context shape:  (17301, 26)
Words shape:  (17301, 26)
Tags shape:  (17301, 1)
Lemmas shape:  (17301, 26, 60)


## Model

In [254]:
EMBEDDING_DIM = 512
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

# train word2vec model
word2vec = gensim.models.Word2Vec(dev_ctx, vector_size=EMBEDDING_DIM, window=10, min_count=1, workers=8)

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_DIM))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec.wv[word]
    except KeyError:
        pass

In [255]:
# Neural network model
# inputs:
#   - context: (batch_size, CTX_DIM * 2) 
#   - tags: encoded tags: (batch_size, 1)
#   - words: encoded words: (batch_size, MAX_WORD_LENGTH)
# outputs:
#  - lemma: encoded lemma: (batch_size, MAX_WORD_LENGTH)

def get_model():
    # context
    context_input = Input(shape=(CTX_DIM * 2,), name="context_input")
    
    context_embedding = Embedding(len(word_tokenizer.word_index) + 1, EMBEDDING_DIM, input_length=CTX_DIM * 2, name="context_embedding", trainable=False, weights=[embedding_weights])(context_input)
    context_embedding = Dense(EMBEDDING_DIM, activation="swish", name="context_embedding_dense")(context_embedding)

    # tags
    tags_input = Input(shape=(1,), name="tags_input")
    tags_embedding = RepeatVector(MAX_WORD_LENGTH)(tags_input)
    tags_embedding = Dense(EMBEDDING_DIM, activation="swish", name="tags_embedding_dense2")(tags_embedding)

    # words
    words_input = Input(shape=(MAX_WORD_LENGTH,), name="words_input")
    words_input = Masking(mask_value=0)(words_input)
    
    words_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_WORD_LENGTH, name="words_embedding", trainable=True)(words_input)
    words_embedding = Dense(EMBEDDING_DIM, activation="swish", name="words_embedding_dense")(words_embedding)

    # combine context, tags and words without using concatenation
    combine = Add()([context_embedding, tags_embedding, words_embedding])

    combine = Dropout(0.5)(combine)
    lstm = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True), name="lstm")(combine)
    lstm = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True), name="lstm2")(lstm)

    # dense layers
    dense1 = Dense(EMBEDDING_DIM, activation="swish", name="dense1")(lstm)
    dense1 = Dropout(0.5)(dense1)
    dense2 = Dense(EMBEDDING_DIM, activation="swish", name="dense2")(dense1)
    dense3 = Dense(EMBEDDING_DIM, activation="swish", name="dense3")(dense2)

    # output
    output = Dense(VOCAB_SIZE, activation="softmax", name="output")(dense3)

    model = Model(inputs=[context_input, tags_input, words_input], outputs=output)
    return model

model = get_model()
model.summary()

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 26)]         0           []                               
                                                                                                  
 tags_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 26)]         0           []                               
                                                                                                  
 context_embedding (Embedding)  (None, 26, 384)      7577088     ['input_11[0][0]']               
                                                                                           

### Lemmatization Accuracy

In [256]:
def accuracy(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=-1)
    y_pred = tf.argmax(y_pred, axis=-1)
    correct_predictions = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy

### Training

In [257]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[accuracy])

# train model
history = model.fit([dev_ctx_e, dev_tags_e, dev_words_e], dev_lemmas_e, epochs=100, batch_size=256, validation_split=0.05)

Epoch 1/100
65/65 [==============================] - 86s 1s/step - loss: 0.9136 - accuracy: 0.0048 - val_loss: 0.6344 - val_accuracy: 0.0000e+00
Epoch 2/100
65/65 [==============================] - 30s 461ms/step - loss: 0.5239 - accuracy: 0.0245 - val_loss: 0.4217 - val_accuracy: 0.0385
Epoch 3/100
65/65 [==============================] - 29s 449ms/step - loss: 0.3211 - accuracy: 0.1782 - val_loss: 0.2371 - val_accuracy: 0.2886
Epoch 4/100
65/65 [==============================] - 26s 396ms/step - loss: 0.1970 - accuracy: 0.3602 - val_loss: 0.1666 - val_accuracy: 0.4778
Epoch 5/100
65/65 [==============================] - 25s 380ms/step - loss: 0.1383 - accuracy: 0.5108 - val_loss: 0.1225 - val_accuracy: 0.6290
Epoch 6/100
65/65 [==============================] - 24s 371ms/step - loss: 0.1140 - accuracy: 0.5715 - val_loss: 0.0909 - val_accuracy: 0.6744
Epoch 7/100
65/65 [==============================] - 24s 366ms/step - loss: 0.0985 - accuracy: 0.5964 - val_loss: 0.0848 - val_accuracy

## Evaluation

In [273]:
# evaluate model
result = model.evaluate([test_ctx_e, test_tags_e, test_words_e], test_lemmas_e)
print("Test loss: ", result[0])
print("Test accuracy: ", result[1])


1957/1957 [==============================] - 202s 103ms/step - loss: 0.0351 - accuracy: 0.9117
Test loss:  0.03508284315466881
Test accuracy:  0.911679208278656
